# Initialization

Test notebook for the C-MAPPS benchmark. Test different MLP architectures. 

First we import the necessary packages and create the global variables.

In [1]:
import math
import numpy as np
import csv
import copy
from scipy import stats
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import custom_scores
from data_handler_CMAPS import CMAPSDataHandler
from tunable_model import SequenceTunableModelRegression
import CMAPSAuxFunctions

from keras.models import Sequential, Model
from keras.layers import Dense, Input, Dropout, Reshape, Conv2D, Flatten, MaxPooling2D
from keras.optimizers import Adam
from keras.callbacks import LearningRateScheduler
from keras import backend as K
from keras import regularizers

Using TensorFlow backend.


# Define architectures

Define each one of the different architectures to be tested.

In [2]:
K.clear_session()  #Clear the previous tensorflow graph

l2_lambda_regularization = 0.20
l1_lambda_regularization = 0.20

def RULmodel_SN_5(input_shape):
    #Create a sequential model
    model = Sequential()
    
    #Add the layers for the model
    model.add(Dense(20, input_dim=input_shape, activation='relu', kernel_initializer='glorot_normal', 
                    kernel_regularizer=regularizers.L1L2(l1_lambda_regularization, l2_lambda_regularization), 
                    name='fc1'))
    model.add(Dense(20, input_dim=input_shape, activation='relu', kernel_initializer='glorot_normal', 
                    kernel_regularizer=regularizers.L1L2(l1_lambda_regularization, l2_lambda_regularization), 
                    name='fc2'))
    model.add(Dense(1, activation='linear', name='out'))
    
    return model

In [3]:
def get_compiled_model(model_def, shape, model_type='ann'):

    #Shared parameters for the models
    optimizer = Adam(lr=0, beta_1=0.5)
    lossFunction = "mean_squared_error"
    metrics = ["mse"]
    model = None

    #Create and compile the models

    if model_type=='ann':
        model = model_def(shape)
        model.compile(optimizer = optimizer, loss = lossFunction, metrics = metrics)
    elif model_type=='lstm':
        """
        nFeatures = len(selected_features)
        shapeLSTM = (window_size, nFeatures)
        model = RULmodel_LSTM(shape)
        model.compile(optimizer = optimizer, loss = lossFunction, metrics = metrics)"""
    else:
        pass

    return model

In [4]:
#Define the usable models for this notebook

models = {'shallow-20':RULmodel_SN_5}
import tensorflow as tf
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

# Process Data

In [5]:
features = ['T2', 'T24', 'T30', 'T50', 'P2', 'P15', 'P30', 'Nf', 'Nc', 'epr', 'Ps30', 'phi', 'NRf', 'NRc', 
                     'BPR', 'farB', 'htBleed', 'Nf_dmd', 'PCNfR_dmd', 'W31', 'W32']
selected_indices = np.array([2, 3, 4, 7, 8, 9, 11, 12, 13, 14, 15, 17, 20, 21])
selected_features = list(features[i] for i in selected_indices-1)
data_folder = '../CMAPSSData'

window_size = 30
window_stride = 1
max_rul = 128

min_max_scaler = MinMaxScaler(feature_range=(-1, 1))

dHandler_cmaps = CMAPSDataHandler(data_folder, 1, selected_features, max_rul, 
                                  window_size, window_stride)

# Build the model

In [6]:
optimizer = Adam(lr=0, beta_1=0.5)
lossFunction = "mean_squared_error"
metrics = ["mse"]


#Create and compile the models
nFeatures = len(selected_features)
shapeSN = nFeatures*window_size
modelRULSN = get_compiled_model(models['shallow-20'], shapeSN, model_type='ann')

tModel = SequenceTunableModelRegression('ModelRUL_SN_5', modelRULSN, lib_type='keras', data_handler=dHandler_cmaps)
tModel.data_scaler = min_max_scaler

# Load Data

In [7]:
#tModel.data_handler.data_scaler = min_max_scaler

tModel.data_handler.sequence_length = 30
#tModel.data_handler.sequence_length = maxWindowSize[datasetNumber]
tModel.data_handler.sequence_stride = 1
tModel.data_handler.max_rul = 128

tModel.load_data(unroll=True, verbose=1, cross_validation_ratio=0)
tModel.print_data()

Loading data for dataset 1 with window_size of 30, stride of 1 and maxRUL of 128. Cros-Validation ratio 0
Loading data from file and computing dataframes
Printing shapes

Training data (X, y)
(17731, 420)
(17731,)
Testing data (X, y)
(100, 420)
(100,)
Printing first 5 elements

Training data (X, y)
[[-0.58075601 -0.0455243  -0.27982732 ... -0.81818182  0.43307087
   0.4679733 ]
 [-0.35395189  0.0629156  -0.18014129 ... -0.45454545  0.25984252
   0.25294702]
 [-0.21649485 -0.13299233 -0.13854003 ... -0.45454545  0.38582677
   0.72049425]
 [-0.21649485 -0.39897698 -0.2299843  ... -0.45454545  0.08661417
   0.29640676]
 [-0.20274914 -0.39590793 -0.05926217 ... -0.45454545  0.05511811
   0.17880983]]
[128. 128. 128. 128. 128.]
Testing data (X, y)
[[-0.65635739 -0.10946292 -0.48312402 ... -0.27272727  0.05511811
   0.30947309]
 [ 0.03780069 -0.07365729 -0.27629513 ... -0.63636364  0.05511811
   0.04416986]
 [ 0.13402062 -0.08644501  0.038854   ...  0.09090909  0.24409449
   0.07882403]
 [-0

# Test on dataset 1

In [8]:
iterations = 10
tModel.epochs = 200
lrate = LearningRateScheduler(CMAPSAuxFunctions.step_decay)
num_features = len(selected_features)

windowSize = 30
windowStride = 1
constRul = 128

file = open("ResultsDataset1.csv", "w")
csvfile = csv.writer(file, lineterminator='\n')

for key, model_def in models.items():
    
    print("For model "+str(key))
    #file.write("For model "+str(key)+'\n\n')
    
    for i in range(1,2):

        dataset = i
        print("Computing for dataset "+str(i))
        #file.write("Computing for dataset "+str(i)+'\n\n')
        
        tempScoresRMSE = np.zeros((iterations,1))
        tempScoresRHS = np.zeros((iterations,1))
        tempTime = np.zeros((iterations,1))
        
        input_shape = windowSize*num_features #For simple ANN
        
        tModel.data_handler.change_dataset(i)
        tModel.data_handler.sequence_length = windowSize
        tModel.data_handler.sequence_stride = windowStride
        tModel.data_handler.max_rul = constRul
        tModel.load_data(unroll=True, verbose=0, cross_validation_ratio=0)
        #tModel.print_data()

        for j in range(iterations):

            #Model needs to be recompiled everytime since they are different runs so weights should be reinit
            model = get_compiled_model(model_def, input_shape, model_type='ann')

            tModel.change_model(key, model, 'keras')
            tModel.train_model(learningRate_scheduler=lrate, verbose=0)
            tModel.evaluate_model(['rhs', 'rmse'], round=2)
            #print("scores")
            
            #print(j)

            cScores = tModel.scores
            rmse = math.sqrt(cScores['score_1'])
            rmse2 = cScores['rmse']
            rhs = cScores['rhs']
            time = tModel.train_time
            
            tempScoresRMSE[j] = rmse2
            tempScoresRHS[j] = rhs
            tempTime[j] = time

        print("Results for model " + key)
    
        print(stats.describe(tempScoresRMSE))
        print(stats.describe(tempScoresRHS))
        print(stats.describe(tempTime))
            
        tempScoresRMSE = np.reshape(tempScoresRMSE, (iterations,))
        tempScoresRHS = np.reshape(tempScoresRHS, (iterations,))
        tempTime = np.reshape(tempTime, (iterations,))
        csvfile.writerow(tempScoresRMSE)
        csvfile.writerow(tempScoresRHS)
        csvfile.writerow(tempTime)
        
file.close()

For model shallow-20
Computing for dataset 1
Loading data from file and computing dataframes
Epoch 1/200
17731/17731 [==============================] - 1s 51us/step - loss: 8156.3005 - mean_squared_error: 8043.7277
Epoch 2/200
17731/17731 [==============================] - 0s 7us/step - loss: 5025.4470 - mean_squared_error: 4904.2545
Epoch 3/200
17731/17731 [==============================] - 0s 7us/step - loss: 1752.2857 - mean_squared_error: 1611.9952
Epoch 4/200
17731/17731 [==============================] - 0s 7us/step - loss: 949.0844 - mean_squared_error: 808.7970
Epoch 5/200
17731/17731 [==============================] - 0s 7us/step - loss: 688.1716 - mean_squared_error: 547.3473
Epoch 6/200
17731/17731 [==============================] - 0s 7us/step - loss: 592.3429 - mean_squared_error: 447.4370
Epoch 7/200
17731/17731 [==============================] - 0s 7us/step - loss: 545.6114 - mean_squared_error: 398.9398
Epoch 8/200
17731/17731 [==============================] - 0s 7us/s

17731/17731 [==============================] - 0s 7us/step - loss: 279.3087 - mean_squared_error: 200.7191
Epoch 69/200
17731/17731 [==============================] - 0s 7us/step - loss: 279.8561 - mean_squared_error: 201.8130
Epoch 70/200
17731/17731 [==============================] - 0s 7us/step - loss: 278.4199 - mean_squared_error: 200.7916
Epoch 71/200
17731/17731 [==============================] - 0s 7us/step - loss: 276.2264 - mean_squared_error: 199.2501
Epoch 72/200
17731/17731 [==============================] - 0s 7us/step - loss: 275.9449 - mean_squared_error: 199.3996
Epoch 73/200
17731/17731 [==============================] - 0s 7us/step - loss: 274.8394 - mean_squared_error: 198.8561
Epoch 74/200
17731/17731 [==============================] - 0s 7us/step - loss: 274.5980 - mean_squared_error: 199.0794
Epoch 75/200
17731/17731 [==============================] - 0s 7us/step - loss: 274.4949 - mean_squared_error: 199.4550
Epoch 76/200
17731/17731 [===========================

17731/17731 [==============================] - 0s 7us/step - loss: 253.1086 - mean_squared_error: 188.0122
Epoch 137/200
17731/17731 [==============================] - 0s 7us/step - loss: 253.0294 - mean_squared_error: 187.9678
Epoch 138/200
17731/17731 [==============================] - 0s 7us/step - loss: 252.9413 - mean_squared_error: 187.9160
Epoch 139/200
17731/17731 [==============================] - 0s 7us/step - loss: 252.8152 - mean_squared_error: 187.8096
Epoch 140/200
17731/17731 [==============================] - 0s 8us/step - loss: 252.7762 - mean_squared_error: 187.8104
Epoch 141/200
17731/17731 [==============================] - 0s 7us/step - loss: 252.7173 - mean_squared_error: 187.7702
Epoch 142/200
17731/17731 [==============================] - 0s 7us/step - loss: 252.6744 - mean_squared_error: 187.7685
Epoch 143/200
17731/17731 [==============================] - 0s 7us/step - loss: 252.5697 - mean_squared_error: 187.7105
Epoch 144/200
17731/17731 [===================

17731/17731 [==============================] - 0s 8us/step - loss: 1960.7286 - mean_squared_error: 1808.3381
Epoch 4/200
17731/17731 [==============================] - 0s 7us/step - loss: 1044.2462 - mean_squared_error: 888.3746
Epoch 5/200
17731/17731 [==============================] - 0s 7us/step - loss: 725.2589 - mean_squared_error: 569.3851
Epoch 6/200
17731/17731 [==============================] - 0s 7us/step - loss: 620.2904 - mean_squared_error: 463.3560
Epoch 7/200
17731/17731 [==============================] - 0s 7us/step - loss: 576.3449 - mean_squared_error: 418.7160
Epoch 8/200
17731/17731 [==============================] - 0s 7us/step - loss: 546.9692 - mean_squared_error: 388.1379
Epoch 9/200
17731/17731 [==============================] - 0s 8us/step - loss: 520.5368 - mean_squared_error: 359.9676
Epoch 10/200
17731/17731 [==============================] - 0s 8us/step - loss: 490.8659 - mean_squared_error: 327.9132
Epoch 11/200
17731/17731 [==============================

17731/17731 [==============================] - 0s 7us/step - loss: 284.9109 - mean_squared_error: 204.5068
Epoch 72/200
17731/17731 [==============================] - 0s 8us/step - loss: 283.7315 - mean_squared_error: 203.8978
Epoch 73/200
17731/17731 [==============================] - 0s 7us/step - loss: 283.2482 - mean_squared_error: 203.9061
Epoch 74/200
17731/17731 [==============================] - 0s 7us/step - loss: 283.5419 - mean_squared_error: 204.6379
Epoch 75/200
17731/17731 [==============================] - 0s 7us/step - loss: 280.9983 - mean_squared_error: 202.5953
Epoch 76/200
17731/17731 [==============================] - 0s 7us/step - loss: 279.9524 - mean_squared_error: 202.0394
Epoch 77/200
17731/17731 [==============================] - 0s 7us/step - loss: 279.1957 - mean_squared_error: 201.7899
Epoch 78/200
17731/17731 [==============================] - 0s 7us/step - loss: 278.4550 - mean_squared_error: 201.4373
Epoch 79/200
17731/17731 [===========================

...

17731/17731 [==============================] - 0s 8us/step - loss: 246.7787 - mean_squared_error: 179.1573
Epoch 90/200
17731/17731 [==============================] - 0s 8us/step - loss: 245.8116 - mean_squared_error: 178.4029
Epoch 91/200
17731/17731 [==============================] - 0s 8us/step - loss: 245.7914 - mean_squared_error: 178.7391
Epoch 92/200
17731/17731 [==============================] - 0s 8us/step - loss: 244.8095 - mean_squared_error: 178.0175
Epoch 93/200
17731/17731 [==============================] - 0s 7us/step - loss: 245.3740 - mean_squared_error: 178.8180
Epoch 94/200
17731/17731 [==============================] - 0s 7us/step - loss: 244.9096 - mean_squared_error: 178.5969
Epoch 95/200
17731/17731 [==============================] - 0s 8us/step - loss: 243.9411 - mean_squared_error: 177.9301
Epoch 96/200
17731/17731 [==============================] - 0s 7us/step - loss: 244.3971 - mean_squared_error: 178.5725
Epoch 97/200
17731/17731 [===========================

17731/17731 [==============================] - 0s 7us/step - loss: 236.7462 - mean_squared_error: 174.2410
Epoch 157/200
17731/17731 [==============================] - 0s 7us/step - loss: 236.6578 - mean_squared_error: 174.1722
Epoch 158/200
17731/17731 [==============================] - 0s 8us/step - loss: 236.6314 - mean_squared_error: 174.1780
Epoch 159/200
17731/17731 [==============================] - 0s 8us/step - loss: 236.6390 - mean_squared_error: 174.2167
Epoch 160/200
17731/17731 [==============================] - 0s 8us/step - loss: 236.5122 - mean_squared_error: 174.1308
Epoch 161/200
17731/17731 [==============================] - 0s 8us/step - loss: 236.4491 - mean_squared_error: 174.1086
Epoch 162/200
17731/17731 [==============================] - 0s 8us/step - loss: 236.3967 - mean_squared_error: 174.1149
Epoch 163/200
17731/17731 [==============================] - 0s 8us/step - loss: 236.4352 - mean_squared_error: 174.1599
Epoch 164/200
17731/17731 [===================

17731/17731 [==============================] - 0s 8us/step - loss: 353.4797 - mean_squared_error: 220.7673
Epoch 24/200
17731/17731 [==============================] - 0s 8us/step - loss: 350.2489 - mean_squared_error: 220.1573
Epoch 25/200
17731/17731 [==============================] - 0s 7us/step - loss: 346.2686 - mean_squared_error: 218.8173
Epoch 26/200
17731/17731 [==============================] - 0s 8us/step - loss: 343.3378 - mean_squared_error: 218.1041
Epoch 27/200
17731/17731 [==============================] - 0s 8us/step - loss: 340.7883 - mean_squared_error: 217.9647
Epoch 28/200
17731/17731 [==============================] - 0s 8us/step - loss: 336.9823 - mean_squared_error: 216.1620
Epoch 29/200
17731/17731 [==============================] - 0s 8us/step - loss: 334.5136 - mean_squared_error: 215.8858
Epoch 30/200
17731/17731 [==============================] - 0s 8us/step - loss: 331.0932 - mean_squared_error: 214.4150
Epoch 31/200
17731/17731 [===========================

17731/17731 [==============================] - 0s 8us/step - loss: 263.6711 - mean_squared_error: 189.5365
Epoch 92/200
17731/17731 [==============================] - 0s 7us/step - loss: 263.7532 - mean_squared_error: 189.9275
Epoch 93/200
17731/17731 [==============================] - 0s 8us/step - loss: 262.2004 - mean_squared_error: 188.6900
Epoch 94/200
17731/17731 [==============================] - 0s 7us/step - loss: 262.8177 - mean_squared_error: 189.5466
Epoch 95/200
17731/17731 [==============================] - 0s 8us/step - loss: 261.3815 - mean_squared_error: 188.3682
Epoch 96/200
17731/17731 [==============================] - 0s 8us/step - loss: 262.8294 - mean_squared_error: 190.0323
Epoch 97/200
17731/17731 [==============================] - 0s 8us/step - loss: 262.0860 - mean_squared_error: 189.5786
Epoch 98/200
17731/17731 [==============================] - 0s 8us/step - loss: 260.5281 - mean_squared_error: 188.2816
Epoch 99/200
17731/17731 [===========================

17731/17731 [==============================] - 0s 8us/step - loss: 252.9063 - mean_squared_error: 183.8063
Epoch 159/200
17731/17731 [==============================] - 0s 8us/step - loss: 252.8590 - mean_squared_error: 183.7901
Epoch 160/200
17731/17731 [==============================] - 0s 7us/step - loss: 252.7728 - mean_squared_error: 183.7581
Epoch 161/200
17731/17731 [==============================] - 0s 7us/step - loss: 252.6969 - mean_squared_error: 183.7149
Epoch 162/200
17731/17731 [==============================] - 0s 8us/step - loss: 252.6276 - mean_squared_error: 183.7125
Epoch 163/200
17731/17731 [==============================] - 0s 8us/step - loss: 252.4918 - mean_squared_error: 183.6133
Epoch 164/200
17731/17731 [==============================] - 0s 8us/step - loss: 252.4983 - mean_squared_error: 183.6599
Epoch 165/200
17731/17731 [==============================] - 0s 8us/step - loss: 252.3347 - mean_squared_error: 183.5194
Epoch 166/200
17731/17731 [===================

# Test on all Datasets

In [9]:
datasets = [1,2,3,4]
iterations = 10
tModel.epochs = 200
lrate = LearningRateScheduler(CMAPSAuxFunctions.step_decay)
scores ={1:[], 2:[], 3:[], 4:[]}
window_sizes = {1:30,2:20,3:30,4:18}
strides = {1:1,2:2,3:1,4:2}
max_ruls = {1:128, 2:134, 3:128, 4:134}
num_features = len(selected_features)

input_shape = None

#For each model
for key, model_def in models.items():
    file = open("results/MLP/ResultsDatasets_ScalerAtEnd"+key+".csv", "w")
    csvfile = csv.writer(file, lineterminator='\n')
    
    print("Generating statistics for model " + key)

    #For each dataset
    for i in range(1,5):
        
        print("Working on dataset " + str(i))
        
        tempScoresRMSE = np.zeros((iterations,1))
        tempScoresRHS = np.zeros((iterations,1))
        tempTime = np.zeros((iterations,1))
        
        input_shape = window_sizes[i]*num_features #For simple ANN
        #input_shape = (window_sizes,num_features) #For RNN
        
        tModel.data_handler.change_dataset(i)
        tModel.data_handler.sequence_length = window_sizes[i]
        tModel.data_handler.sequence_stride = strides[i]
        tModel.data_handler.max_rul = max_ruls[i]
        tModel.load_data(unroll=True, verbose=0, cross_validation_ratio=0)
        #tModel.print_data()
        
        #tModel.print_data()
        
        for j in range(iterations):

            #Model needs to be recompiled everytime since they are different runs so weights should be reinit
            model = get_compiled_model(model_def, input_shape, model_type='ann')

            tModel.change_model(key, model, 'keras')
            tModel.train_model(learningRate_scheduler=lrate, verbose=0)
            tModel.evaluate_model(['rhs', 'rmse'], round=2)
            #print("scores")
            
            #print(j)

            cScores = tModel.scores
            rmse = math.sqrt(cScores['score_1'])
            rmse2 = cScores['rmse']
            rhs = cScores['rhs']
            time = tModel.train_time
            
            tempScoresRMSE[j] = rmse2
            tempScoresRHS[j] = rhs
            tempTime[j] = time
            
        print("Results for model " + key)
    
        print(stats.describe(tempScoresRMSE))
        print(stats.describe(tempScoresRHS))
        print(stats.describe(tempTime))
            
        tempScoresRMSE = np.reshape(tempScoresRMSE, (iterations,))
        tempScoresRHS = np.reshape(tempScoresRHS, (iterations,))
        tempTime = np.reshape(tempTime, (iterations,))
        csvfile.writerow(tempScoresRMSE)
        csvfile.writerow(tempScoresRHS)
        csvfile.writerow(tempTime)
    
    file.close()

Generating statistics for model shallow-20
Working on dataset 1
Loading data from file and computing dataframes
100/100 [==============================] - 0s 2ms/step
Results for model shallow-20
DescribeResult(nobs=10, minmax=(array([14.69217479]), array([15.12150786])), mean=array([14.95405457]), variance=array([0.01583542]), skewness=array([-0.87059511]), kurtosis=array([0.07823726]))
DescribeResult(nobs=10, minmax=(array([3.64611613]), array([4.05926767])), mean=array([3.87405373]), variance=array([0.01460501]), skewness=array([-0.34469135]), kurtosis=array([-0.49364936]))
DescribeResult(nobs=10, minmax=(array([26.90017367]), array([28.19250369])), mean=array([27.51810115]), variance=array([0.2190123]), skewness=array([0.09686868]), kurtosis=array([-1.30020197]))
Working on dataset 2
Loading data from file and computing dataframes
259/259 [==============================] - 0s 1ms/step
Results for model shallow-20
DescribeResult(nobs=10, minmax=(array([30.18827664]), array([30.62325